# Code used to visualize the _Looking for Lucy_ post

In [111]:
# Import modules
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
from scipy.spatial import Delaunay

### Loading and displaying a 3D mesh

In [195]:
# Load data
mesh = o3d.io.read_triangle_mesh("../data/ship.ply")
tri = np.asarray(mesh.vertices)
x = tri[:, 0] * 100
y = tri[:, 1] * 100
z = tri[:, 2] * 100
simplices = np.asarray(mesh.triangles)

In [305]:
# Plot figure
fig = ff.create_trisurf(x=x, y=y, z=z, simplices=simplices, plot_edges=False, show_colorbar=False)
fig.update_traces(hoverinfo='none')

fig.add_trace(
    go.Scatter3d(x=[-10000], y=[0], z=[-6000],
                 mode='markers',
                 hovertext='Unlikely',
                 hoverinfo='text',
                 showlegend=False)
)
fig.add_trace(
    go.Scatter3d(x=[0], y=[0], z=[-3500],
                 mode='markers',
                 hovertext='Likely',
                 hoverinfo='text',
                 showlegend=False)
)

camera = dict(
    eye=dict(x=-1.6, y=-1.6, z=1),
    center=dict(x=0.4, y=0, z=-0.3))

fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False),
                    aspectmode='data'),
                  hoverlabel=dict(font_size=18, font_family="Courier New"),
                  width=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0),
                  scene_camera=camera)

In [306]:
# Save figure
pio.write_html(fig,
               file='../_includes/figures/ship.html',
               full_html=False,
               # include_mathjax='cdn',
               include_plotlyjs='cdn')

In [132]:
# Our 2-dimensional distribution will be over variables X and Y
N = 60
x = np.linspace(-3.5, 3.5, N)
y = np.linspace(-3.5, 3.5, N)
X, Y = np.meshgrid(x, y)


# Mean vector and covariance matrix
mu = np.array([0., 0.])
Sigma = np.array([[ 2.5 , 1], [1,  2.5]])

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

def multivariate_gaussian(pos, mu, Sigma):
    """Return the multivariate Gaussian distribution on array pos.

    pos is an array constructed by packing the meshed arrays of variables
    x_1, x_2, x_3, ..., x_k into its _last_ dimension.

    """

    n = mu.shape[0]
    Sigma_det = np.linalg.det(Sigma)
    Sigma_inv = np.linalg.inv(Sigma)
    N = np.sqrt((2*np.pi)**n * Sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

# The distribution on the variables X, Y packed into pos.
Z = multivariate_gaussian(pos, mu, Sigma)
fig = go.Figure(data=go.Surface(x=X, y=Y, z=Z))
fig.update_traces(contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True),
                  showscale=False, opacity=0.6)
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False),
                    zaxis=dict(visible=False, range=[0, 0.07]),
                    domain=dict(y=[0.1, 1])),
                  height=700,
                  margin=dict(r=0, l=0, b=0, t=0, pad=0))